In [ ]:
# https://www.imdb.com/title/tt10192406/fullcredits?ref_=tt_cl_sm

In [ ]:
# TODO: Import ./data/oscar_movies.csv and retrieve 'id' column
# TODO: From the resulting dataframe, retrieve top 5 ids in the list and generate a for loop
# TODO: In the for loop, run a function "retrieve_cast_details"
# TODO: Create a function "retrieve_cast_details" starting with inserting 'id' into "https://www.imdb.com/title/{id}/fullcredits" and prepare for web scraping
# TODO: In the fuction, retrieve the members for the followng roles - Director, Writing Credits, Cast Actors, Producer, Casting Director, Casting Department

-----
-----
Logic

Loop through the list of all oscar winning and nominated movies.

For each movie, iterate through the following
- Director
```
<h4 name="director" id="director" class="dataHeaderWithBorder">Directed by</h4>
```
- Writer
```
<h4 name="writer" id="writer" class="dataHeaderWithBorder">Writing Credits</h4>
```
- Cast
```
In <table class="cast_list">, between <td colspan="4" class="castlist_label"></td> and <td colspan="4" class="castlist_label">Rest of cast listed alphabetically:</td>
```
- Producing
```
<h4 name="producer" id="producer" class="dataHeaderWithBorder">
```
- Casting & Casting Department
```
<h4 name="casting_director" id="casting_director" class="dataHeaderWithBorder">
```
```
<h4 name="casting_department" id="casting_department" class="dataHeaderWithBorder">…</h4>
``` 


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def retrieve_cast_details(id):
    # prepare URL
    url = f"https://www.imdb.com/title/{id}/fullcredits"
    
    # send GET request
    response = requests.get(url)
    
    # parse HTML response
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # collect the names of different roles
    roles = ['director', 'writer', 'producer', 'casting_director', 'casting_department']
    
    # dictionary to store details
    cast_details = {}
    
    # iterate over roles and scrape the webpage
    for role in roles:
        role_section = soup.find('h4', attrs={'name': role, 'id': role, 'class': 'dataHeaderWithBorder'})
        if role_section:
            # retrieve next sibling element which contains names
            names_element = role_section.find_next_sibling('table')
            if names_element:
                # extract and clean names
                names = [name.get_text(strip=True) for name in names_element.find_all('td', class_='name') if name.get_text(strip=True) not in ['...', '']]
                cast_details[role] = names
    
    # special case for 'cast' role
    cast_section = soup.find('table', class_='cast_list')
    if cast_section:
        cast_names = [actor.get_text(strip=True) for actor in cast_section.find_all('td', class_=lambda x: x != 'character') if actor.get_text(strip=True) not in ['...', '']]
        cast_details['cast'] = cast_names
    
    return cast_details

# read CSV file
df = pd.read_csv('./data/oscar_movies.csv')

# retrieve 'id' column
ids = df['id']

# get top 5 ids
top_ids = ids[:5]

# iterate over ids and retrieve cast details
for id in top_ids:
    cast_details = retrieve_cast_details(id)
    print(f"ID: {id}")
    print(cast_details)


ID: tt0015864
{'director': ['Charles Chaplin'], 'writer': ['Charles Chaplin'], 'producer': ['Charles Chaplin'], 'cast': ['Charles Chaplin', 'Mack Swain', 'Tom Murray', 'Henry Bergman', 'Malcolm Waite', 'Georgia Hale', 'Rest of cast listed alphabetically:', 'Jack Adams', 'Frank Aderias', 'Leona Aderias', 'Lillian Adrian', 'Sam Allen', 'Claude Anderson', 'Harry Arras', 'Albert Austin', 'Marta Belfort', 'William Bell', 'Francis Bernhardt', 'F.J. Beuaregard', 'E. Blumenthal', 'William Bradford', 'George Brock', 'Pete Brogan', 'John Brown', 'William Butler', 'Cecile Cameron', 'R. Campbell', 'Leland Carr', 'H.C. Chisholm', 'Harry Coleman', 'Heinie Conklin', 'Rebecca Conroy', 'Dorothy Crane', 'James Darby', 'Harry De More', 'Kay Deslys', 'Jimmy Dime', 'W.S. Dobson', 'John Eagown', 'Aaron Edward', 'E. Espinosa', 'Leon Farey', 'M. Farrell', 'Richard Foley', 'Charles Force', 'J.C. Fowler', 'Al Ernest Garcia', 'Inez Gomez', 'Sid Grauman', 'Lita Grey', 'Ray Grey', 'F.F. Guenste', 'William Hackett'

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def retrieve_cast_details(id):
    # Construct URL
    url = f"https://www.imdb.com/title/{id}/fullcredits"

    # Send GET request
    response = requests.get(url)

    # Parse HTML response
    soup = BeautifulSoup(response.text, 'html.parser')

    # Define the roles to extract
    roles = ['director', 'writer', 'producer', 'casting_director', 'casting_department']

    # Initialize dictionary to hold details
    cast_details = {}

    # Loop over roles and scrape the webpage
    for role in roles:
        role_section = soup.find('h4', attrs={'name': role, 'id': role, 'class': 'dataHeaderWithBorder'})
        if role_section:
            # Retrieve next sibling element which contains names
            names_element = role_section.find_next_sibling('table')
            if names_element:
                # Extract and sanitize names
                names = [name.get_text(strip=True) for name in names_element.find_all('td', class_='name') if name.get_text(strip=True) not in ['...', '']]
                cast_details[role] = names

    # Handle 'cast' role separately
    cast_section = soup.find('table', class_='cast_list')
    if cast_section:
        cast_names = [actor.get_text(strip=True) for actor in cast_section.find_all('td', class_=lambda x: x != 'character') if actor.get_text(strip=True) not in ['...', '']]
        # Split the cast_names list into major and minor cast based on the specific string
        if 'Rest of cast listed alphabetically:' in cast_names:
            split_index = cast_names.index('Rest of cast listed alphabetically:')
            cast_details['cast_major'] = cast_names[:split_index]
            cast_details['cast_minor'] = cast_names[split_index+1:]
        else:
            cast_details['cast_major'] = cast_names
            cast_details['cast_minor'] = []

    return cast_details

# Read CSV file
df = pd.read_csv('./data/oscar_movies.csv')

# Extract 'id' column
ids = df['id']

# Get first 5 ids
top_ids = ids[:5]

# Loop over ids and retrieve cast details
for id in top_ids:
    cast_details = retrieve_cast_details(id)
    print(f"ID: {id}")
    print(cast_details)


ID: tt0015864
{'director': ['Charles Chaplin'], 'writer': ['Charles Chaplin'], 'producer': ['Charles Chaplin'], 'cast_major': ['Charles Chaplin', 'Mack Swain', 'Tom Murray', 'Henry Bergman', 'Malcolm Waite', 'Georgia Hale'], 'cast_minor': ['Jack Adams', 'Frank Aderias', 'Leona Aderias', 'Lillian Adrian', 'Sam Allen', 'Claude Anderson', 'Harry Arras', 'Albert Austin', 'Marta Belfort', 'William Bell', 'Francis Bernhardt', 'F.J. Beuaregard', 'E. Blumenthal', 'William Bradford', 'George Brock', 'Pete Brogan', 'John Brown', 'William Butler', 'Cecile Cameron', 'R. Campbell', 'Leland Carr', 'H.C. Chisholm', 'Harry Coleman', 'Heinie Conklin', 'Rebecca Conroy', 'Dorothy Crane', 'James Darby', 'Harry De More', 'Kay Deslys', 'Jimmy Dime', 'W.S. Dobson', 'John Eagown', 'Aaron Edward', 'E. Espinosa', 'Leon Farey', 'M. Farrell', 'Richard Foley', 'Charles Force', 'J.C. Fowler', 'Al Ernest Garcia', 'Inez Gomez', 'Sid Grauman', 'Lita Grey', 'Ray Grey', 'F.F. Guenste', 'William Hackett', 'Mildred Hall',

In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def retrieve_cast_details(id):
    # Construct URL
    url = f"https://www.imdb.com/title/{id}/fullcredits"

    # Send GET request
    response = requests.get(url)

    # Parse HTML response
    soup = BeautifulSoup(response.text, 'html.parser')

    # Define the roles to extract
    roles = ['director', 'writer', 'producer', 'casting_director', 'casting_department']

    # Initialize dictionary to hold details
    cast_details = {}

    # Loop over roles and scrape the webpage
    for role in roles:
        role_section = soup.find('h4', attrs={'name': role, 'id': role, 'class': 'dataHeaderWithBorder'})
        if role_section:
            # Retrieve next sibling element which contains names
            names_element = role_section.find_next_sibling('table')
            if names_element:
                # Extract and sanitize names
                name_ids = []
                for name in names_element.find_all('a', href=True):
                    match = re.search('/name/(.*?)/', name['href'])
                    if match:
                        name_ids.append(match.group(1))
                cast_details[role] = name_ids

    # Handle 'cast' role separately
    cast_section = soup.find('table', class_='cast_list')
    if cast_section:
        cast_ids = []
        cast_names = []
        for actor in cast_section.find_all('td', class_=lambda x: x != 'character'):
            actor_name = actor.get_text(strip=True)
            if actor_name not in ['...', '']:
                cast_names.append(actor_name)
                actor_link = actor.find('a')
                if actor_link and 'href' in actor_link.attrs:
                    match = re.search('/name/(.*?)/', actor_link['href'])
                    if match:
                        cast_ids.append(match.group(1))

        if 'Rest of cast listed alphabetically:' in cast_names:
            split_index = cast_names.index('Rest of cast listed alphabetically:')
            cast_details['cast_major'] = cast_ids[:split_index]
            cast_details['cast_minor'] = cast_ids[split_index+1:]
        else:
            cast_details['cast_major'] = cast_ids
            cast_details['cast_minor'] = []

    return cast_details

# Read CSV file
df = pd.read_csv('./data/oscar_movies.csv')

# Extract 'id' column
ids = df['id']

# Get first 5 ids
top_ids = ids[:5]

# Loop over ids and retrieve cast details
for id in top_ids:
    cast_details = retrieve_cast_details(id)
    print(f"ID: {id}")
    print(cast_details)


ID: tt0015864
{'director': ['nm0000122'], 'writer': ['nm0000122'], 'producer': ['nm0000122'], 'cast_major': ['nm0000122', 'nm0841501', 'nm0615306', 'nm0074788', 'nm0906618', 'nm0354913'], 'cast_minor': ['nm1027682', 'nm1027683', 'nm0012447', 'nm0021025', 'nm1035034', 'nm0037161', 'nm0042317', 'nm1028059', 'nm1033941', 'nm1028111', 'nm1028130', 'nm1035124', 'nm1033950', 'nm1033954', 'nm1028362', 'nm9261438', 'nm1033965', 'nm0131521', 'nm1028477', 'nm0139782', 'nm1035210', 'nm0171047', 'nm0174692', 'nm1028723', 'nm1036418', 'nm1035268', 'nm0210468', 'nm0221353', 'nm0227243', 'nm1029091', 'nm1029190', 'nm1035329', 'nm1029312', 'nm1029360', 'nm1029358', 'nm1034029', 'nm0285443', 'nm0288721', 'nm0305087', 'nm0327021', 'nm0336138', 'nm0340614', 'nm0336894', 'nm0345853', 'nm0352509', 'nm1036540', 'nm1034056', 'nm1029958', 'nm0366305', 'nm1029989', 'nm1035490', 'nm0371746', 'nm0380605', 'nm1030090', 'nm0392429', 'nm1036573', 'nm0403156', 'nm1034084', 'nm0421368', 'nm0426647', 'nm1034096', 'nm0

In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def retrieve_cast_details(id):
    url = f"https://www.imdb.com/title/{id}/fullcredits"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    roles = ['director', 'writer', 'producer', 'casting_director', 'casting_department']
    cast_details = {}
    for role in roles:
        role_section = soup.find('h4', attrs={'name': role, 'id': role, 'class': 'dataHeaderWithBorder'})
        if role_section:
            names_element = role_section.find_next_sibling('table')
            if names_element:
                name_ids = []
                for name in names_element.find_all('a', href=True):
                    match = re.search('/name/(.*?)/', name['href'])
                    if match:
                        name_ids.append(match.group(1))
                cast_details[role] = name_ids

    cast_section = soup.find('table', class_='cast_list')
    if cast_section:
        cast_ids = []
        cast_names = []
        rest_of_cast = False
        for actor in cast_section.find_all('td', class_=lambda x: x != 'character'):
            actor_name = actor.get_text(strip=True)
            if 'Rest of cast listed alphabetically:' in actor_name:
                rest_of_cast = True
            elif actor_name not in ['...', '']:
                actor_link = actor.find('a')
                if actor_link and 'href' in actor_link.attrs:
                    match = re.search('/name/(.*?)/', actor_link['href'])
                    if match:
                        if rest_of_cast:
                            cast_names.append('Rest of cast listed alphabetically:')
                            rest_of_cast = False
                        cast_ids.append(match.group(1))
                        cast_names.append(actor_name)
        
        if 'Rest of cast listed alphabetically:' in cast_names:
            split_index = cast_names.index('Rest of cast listed alphabetically:')
            cast_details['cast_major'] = cast_ids[:split_index]
            cast_details['cast_minor'] = cast_ids[split_index+1:]
        else:
            cast_details['cast_major'] = cast_ids
            cast_details['cast_minor'] = []

    return cast_details

df = pd.read_csv('./data/oscar_movies.csv')
ids = df['id']
top_ids = ids[:5]

for id in top_ids:
    cast_details = retrieve_cast_details(id)
    print(f"ID: {id}")
    print(cast_details)


ID: tt0015864
{'director': ['nm0000122'], 'writer': ['nm0000122'], 'producer': ['nm0000122'], 'cast_major': ['nm0000122', 'nm0841501', 'nm0615306', 'nm0074788', 'nm0906618', 'nm0354913'], 'cast_minor': ['nm1027682', 'nm1027683', 'nm0012447', 'nm0021025', 'nm1035034', 'nm0037161', 'nm0042317', 'nm1028059', 'nm1033941', 'nm1028111', 'nm1028130', 'nm1035124', 'nm1033950', 'nm1033954', 'nm1028362', 'nm9261438', 'nm1033965', 'nm0131521', 'nm1028477', 'nm0139782', 'nm1035210', 'nm0171047', 'nm0174692', 'nm1028723', 'nm1036418', 'nm1035268', 'nm0210468', 'nm0221353', 'nm0227243', 'nm1029091', 'nm1029190', 'nm1035329', 'nm1029312', 'nm1029360', 'nm1029358', 'nm1034029', 'nm0285443', 'nm0288721', 'nm0305087', 'nm0327021', 'nm0336138', 'nm0340614', 'nm0336894', 'nm0345853', 'nm0352509', 'nm1036540', 'nm1034056', 'nm1029958', 'nm0366305', 'nm1029989', 'nm1035490', 'nm0371746', 'nm0380605', 'nm1030090', 'nm0392429', 'nm1036573', 'nm0403156', 'nm1034084', 'nm0421368', 'nm0426647', 'nm1034096', 'nm0

In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def get_name_ids_from_section(section):
    """Extracts and returns name ids from the provided section."""
    name_ids = []
    for name in section.find_all('a', href=True):
        match = re.search('/name/(.*?)/', name['href'])
        if match:
            name_ids.append(match.group(1))
    return name_ids

def get_cast_ids(actor):
    """Extracts and returns actor ids from the provided actor section."""
    actor_link = actor.find('a')
    if actor_link and 'href' in actor_link.attrs:
        match = re.search('/name/(.*?)/', actor_link['href'])
        if match:
            return match.group(1)

def retrieve_cast_details(id):
    url = f"https://www.imdb.com/title/{id}/fullcredits"
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    roles = ['director', 'writer', 'producer', 'casting_director', 'casting_department']
    cast_details = {role: get_name_ids_from_section(soup.find('h4', attrs={'name': role, 'id': role, 'class': 'dataHeaderWithBorder'}).find_next_sibling('table')) 
                    for role in roles if soup.find('h4', attrs={'name': role, 'id': role, 'class': 'dataHeaderWithBorder'})}

    cast_section = soup.find('table', class_='cast_list')
    if cast_section:
        cast_names = [actor.get_text(strip=True) for actor in cast_section.find_all('td', class_=lambda x: x != 'character') if actor.get_text(strip=True) not in ['...', '']]
        cast_ids = [get_cast_ids(actor) for actor in cast_section.find_all('td', class_=lambda x: x != 'character') if actor.get_text(strip=True) not in ['...', '']]
        cast_ids = [id for id in cast_ids if id]  # filter out None values

        split_index = cast_names.index('Rest of cast listed alphabetically:') if 'Rest of cast listed alphabetically:' in cast_names else len(cast_ids)
        cast_details['cast_major'] = cast_ids[:split_index]
        cast_details['cast_minor'] = cast_ids[split_index:] if split_index != len(cast_ids) else []

    return cast_details

df = pd.read_csv('./data/oscar_movies.csv')
for id in df['id'][:5]:
    print(f"ID: {id}")
    print(retrieve_cast_details(id))


ID: tt0015864
{'director': ['nm0000122'], 'writer': ['nm0000122'], 'producer': ['nm0000122'], 'cast_major': ['nm0000122', 'nm0841501', 'nm0615306', 'nm0074788', 'nm0906618', 'nm0354913'], 'cast_minor': ['nm1033900', 'nm1027682', 'nm1027683', 'nm0012447', 'nm0021025', 'nm1035034', 'nm0037161', 'nm0042317', 'nm1028059', 'nm1033941', 'nm1028111', 'nm1028130', 'nm1035124', 'nm1033950', 'nm1033954', 'nm1028362', 'nm9261438', 'nm1033965', 'nm0131521', 'nm1028477', 'nm0139782', 'nm1035210', 'nm0171047', 'nm0174692', 'nm1028723', 'nm1036418', 'nm1035268', 'nm0210468', 'nm0221353', 'nm0227243', 'nm1029091', 'nm1029190', 'nm1035329', 'nm1029312', 'nm1029360', 'nm1029358', 'nm1034029', 'nm0285443', 'nm0288721', 'nm0305087', 'nm0327021', 'nm0336138', 'nm0340614', 'nm0336894', 'nm0345853', 'nm0352509', 'nm1036540', 'nm1034056', 'nm1029958', 'nm0366305', 'nm1029989', 'nm1035490', 'nm0371746', 'nm0380605', 'nm1030090', 'nm0392429', 'nm1036573', 'nm0403156', 'nm1034084', 'nm0421368', 'nm0426647', 'nm1